In [1]:
from copyreg import constructor
from evogym import is_connected, has_actuator, get_full_connectivity, hashable
from evogym.envs import StepsUp, WalkingFlat
import struct
from typing import List, Tuple
import numpy as np
import dill
from pygifsicle import optimize
import imageio
import neat
import os
import neat_evogym

ModuleNotFoundError: No module named 'evogym'

In [4]:
ENV = "StepsUp" # "[WalkingFlat, StepsUp]"
STEPS = 600
STRUCTURE = (5, 5)

In [5]:
winner = None
robot_dict = None
with open("winner.pkl", "rb") as file:
    winner = dill.load(file)
with open("controllers.pkl", "rb") as file:
    robot_dict = dill.load(file)

local_dir = os.path.dirname(".")

config_path = os.path.join(local_dir, 'neat-structure-config')
config_structure = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     config_path)

config_path = os.path.join(local_dir, 'neat-controller-config')
config_controller = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     config_path)



In [6]:
def get_observation(env):
    a = env.get_vel_com_obs("robot")
    b = env.get_pos_com_obs("robot")
    c = env.get_floor_obs("robot", ["ground"], 5)
    return np.concatenate((a, b, c))

def generate_robot(net, structure):
    robot = np.zeros(structure)
    for i in range(structure[0]):
        for j in range(structure[1]):
            input = (i - (structure[0] // 2),
                     j - (structure[1] // 2))
            graph_out = net.activate(input)
            node = np.argmax(graph_out)
            robot[i][j] = node
    return robot

def get_best_controller(robot, robot_dict):
    robot_hash = hashable(robot)
    return robot_dict[robot_hash].population.best_genome

def unhash(robot_hash, structure):
    idx = 0
    robot = np.zeros(structure)
    for i in range(structure[0]):
        for j in range(structure[1]):
            robot[i][j] = robot_hash[idx]
            idx += 1
    return robot

In [7]:
net = neat.nn.FeedForwardNetwork.create(winner, config_structure)
robot = generate_robot(net, STRUCTURE)
controller = get_best_controller(robot, robot_dict)
connections = get_full_connectivity(robot)
r, done = neat_evogym.calculate_reward(ENV, STEPS, robot, connections, controller, config_controller, True)

In [8]:
for robot_hash, controllers in robot_dict.items():
    robot = unhash(robot_hash, STRUCTURE)
    controller = get_best_controller(robot, robot_dict)
    if controller.fitness > 3.5:
        connections = get_full_connectivity(robot)
        r, done = neat_evogym.calculate_reward(ENV, STEPS, robot, connections, controller, config_controller, True)
        print(controller.fitness)

3.6466295375864974
3.8181286404998573
3.5219597187864307
3.516573555788175
